# TARGET: NVICTIM_INTERV

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Partimos del archivo donde se recogen todos los datos de los accidentes, almacenado en nuestro sistema.

In [2]:
df_partida = pd.read_csv('./madrid_accidentes_final_v1.0.csv',encoding='latin1')

df_partida.dtypes

FECHA                       object
RANGO_HORARIO               object
DIA_SEMANA                  object
DISTRITO                    object
LUGAR_ACCIDENTE             object
NUM_CALLE                  float64
NUM_PARTE                   object
Granizo                      int64
Hielo                        int64
Lluvia                       int64
Niebla                       int64
Seco                         int64
Nieve                        int64
supf_Mojada                  int64
supf_Aceite                  int64
supf_Barro                   int64
supf_Grava_Suelta            int64
supf_Hielo                   int64
supf_Seca_Limpia             int64
NUM_VICTIMAS                 int64
TIPO_ACCIDENTE              object
NVICTIM_INTERV              object
MAL_TIEMPO                   int64
MAL_ESTADO_SUELO             int64
num_vehic_implic             int64
n_hombres                    int64
n_mujeres                    int64
n_sex_na                     int64
n_ilesos            

In [3]:
for col in ['RANGO_HORARIO', 'DIA_SEMANA', 'DISTRITO', 'TIPO_ACCIDENTE', 'NVICTIM_INTERV', 'GRAVEDAD_ACCIDENTE']:
    df_partida[col] = df_partida[col].astype('category')

    
df_partida.dtypes  
   

FECHA                        object
RANGO_HORARIO              category
DIA_SEMANA                 category
DISTRITO                   category
LUGAR_ACCIDENTE              object
NUM_CALLE                   float64
NUM_PARTE                    object
Granizo                       int64
Hielo                         int64
Lluvia                        int64
Niebla                        int64
Seco                          int64
Nieve                         int64
supf_Mojada                   int64
supf_Aceite                   int64
supf_Barro                    int64
supf_Grava_Suelta             int64
supf_Hielo                    int64
supf_Seca_Limpia              int64
NUM_VICTIMAS                  int64
TIPO_ACCIDENTE             category
NVICTIM_INTERV             category
MAL_TIEMPO                    int64
MAL_ESTADO_SUELO              int64
num_vehic_implic              int64
n_hombres                     int64
n_mujeres                     int64
n_sex_na                    

## SELECCION DE CAMPOS

In [4]:
df_partida.columns

Index(['FECHA', 'RANGO_HORARIO', 'DIA_SEMANA', 'DISTRITO', 'LUGAR_ACCIDENTE',
       'NUM_CALLE', 'NUM_PARTE', 'Granizo', 'Hielo', 'Lluvia', 'Niebla',
       'Seco', 'Nieve', 'supf_Mojada', 'supf_Aceite', 'supf_Barro',
       'supf_Grava_Suelta', 'supf_Hielo', 'supf_Seca_Limpia', 'NUM_VICTIMAS',
       'TIPO_ACCIDENTE', 'NVICTIM_INTERV', 'MAL_TIEMPO', 'MAL_ESTADO_SUELO',
       'num_vehic_implic', 'n_hombres', 'n_mujeres', 'n_sex_na', 'n_ilesos',
       'n_hgraves', 'n_hleves', 'n_muertos', 'n_lesiv_na', 'conductor_menor',
       'conductor_18_34', 'conductor_35_49', 'conductor_50_mas',
       'conductor_edad_desconoc', 'GRAVEDAD_ACCIDENTE', 'DISTRITO_LAT',
       'DISTRITO_LON'],
      dtype='object')

### Eliminamos con drop los campos no necesarios para el estudio.

Partiendo de todos los datos de los accidentes almacenados en nuestro sistema:

1.- Eliminamos variables descriptivas provenientes de los sistemas operacionales (archivado partes de accidente por la guardia municipal) que no forman parte del estudio:

    - FECHA, DISTRITO, NUM_PARTE, DISTRITO_LAT, DISTRITO_LON
    - LUGAR ACCIDENTE, NUM_CALLE. Señalar que se estudió obtener las coordenadas, paro la tasa de geolocalizados correcta era muy pequeña debido a los datos de partida (gran parte dados como cruce de calles, resto con una proporción de errores (no se devolvia coordenadas)). Imposibilidad de tratamiento automático.
    
    
2.- Debido a lo ya comentado del en relación con el total muy poco número de accidentes con las distintas condiciones adversas climatológicas (nieve, hielo, granizo, etc.) y de estado del suelo (barro, grava suelta, aceite, etc) optamos para el estudio por las variables resumidas 'MAL_TIEMPO' y 'MAL_ESTADO_SUELO', eliminando las mencionadas de las que proceden:

    - 'Granizo', 'Hielo', 'Lluvia', 'Niebla','Seco', 'Nieve'.
    - 'supf_Mojada', 'supf_Aceite', 'supf_Barro', 'supf_Grava_Suelta', 'supf_Hielo', 'supf_Seca_Limpia'.
    

3.- Los datos de número de víctimas registradas con sexo o lesividad no asignados (desconocidos) no aportan nada al estudio (se conservaron por descriptivos), y como número de víctimas ya estan contabilizados en 'NVICTIM_INTERV'. Tampoco interesa el número de ilesos ni el de conductores con edad desconocida. Eliminamos:

    - 'n_sex_na','n_ilesos','n_lesiv_na', 'conductor_edad_desconoc'
    
    
4.- Número de heridos (graves y leves) y número de muertos, así como número de hombres y mujeres entre las víctimas ya estan incluidos en el target NVICTIM_INTERV, son linealmente dependientes (constituyen en cada aspecto la suma del número de víctimas). Las eliminamos:

    - 'n_hombres', 'n_mujeres','n_hgraves', 'n_hleves','n_muertos'.


5.- Por último para quedarnos con sólo el objetivo 'NVICTIM_INTERV' (nº víctimas discretizado), eliminamos 'NUM_VICTIMAS' del que procede. También eliminamos segundo target posible 'GRAVEDAD_ACCIDENTE', atributo calculado a partir del número de heridos de distinta gravedad y número de muertos.

    - 'NUM_VICTIMAS', 'GRAVEDAD_ACCIDENTE'



In [5]:

df_seleccion = df_partida.drop(columns=['FECHA','DISTRITO','LUGAR_ACCIDENTE','NUM_CALLE','NUM_PARTE','DISTRITO_LAT', \
                                        'DISTRITO_LON',\
                                        'n_sex_na','n_ilesos','n_lesiv_na', 'conductor_edad_desconoc', \
                                        'Granizo', 'Hielo', 'Lluvia', 'Niebla','Seco', 'Nieve', 'supf_Mojada', 'supf_Aceite', \
                                        'supf_Barro','supf_Grava_Suelta', 'supf_Hielo', 'supf_Seca_Limpia', \
                                        'n_hombres', 'n_mujeres','n_hgraves', 'n_hleves','n_muertos', \
                                        'NUM_VICTIMAS','GRAVEDAD_ACCIDENTE'])


df_seleccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86817 entries, 0 to 86816
Data columns (total 11 columns):
RANGO_HORARIO       86817 non-null category
DIA_SEMANA          86817 non-null category
TIPO_ACCIDENTE      86817 non-null category
NVICTIM_INTERV      86817 non-null category
MAL_TIEMPO          86817 non-null int64
MAL_ESTADO_SUELO    86817 non-null int64
num_vehic_implic    86817 non-null int64
conductor_menor     86817 non-null int64
conductor_18_34     86817 non-null int64
conductor_35_49     86817 non-null int64
conductor_50_mas    86817 non-null int64
dtypes: category(4), int64(7)
memory usage: 5.0 MB


In [6]:
df_seleccion.head()

,RANGO_HORARIO,DIA_SEMANA,TIPO_ACCIDENTE,NVICTIM_INTERV,MAL_TIEMPO,MAL_ESTADO_SUELO,num_vehic_implic,conductor_menor,conductor_18_34,conductor_35_49,conductor_50_mas
0,DE 00:00 A 7:00,FIN DE SEMANA,COLISION DOBLE,LEVE,1,1,2,0,1,0,1
1,DE 00:00 A 7:00,FIN DE SEMANA,COLISION MULTIPLE,MEDIA/GRAVE,0,1,3,0,3,0,0
2,DE 00:00 A 7:00,FIN DE SEMANA,COLISION DOBLE,LEVE,0,0,2,0,0,2,0
3,DE 00:00 A 7:00,FIN DE SEMANA,COLISION MULTIPLE,MEDIA/GRAVE,0,0,3,0,3,0,0
4,DE 00:00 A 7:00,FIN DE SEMANA,COLISION DOBLE,LEVE,0,0,2,0,1,0,1


### Codificación de las variables categóricas.

In [7]:
df_seleccion.columns

Index(['RANGO_HORARIO', 'DIA_SEMANA', 'TIPO_ACCIDENTE', 'NVICTIM_INTERV',
       'MAL_TIEMPO', 'MAL_ESTADO_SUELO', 'num_vehic_implic', 'conductor_menor',
       'conductor_18_34', 'conductor_35_49', 'conductor_50_mas'],
      dtype='object')

In [8]:
# diferenciamos por tipo


atrib_numericos = ['MAL_TIEMPO', 'MAL_ESTADO_SUELO', 'num_vehic_implic',  \
                   'conductor_menor', 'conductor_18_34', 'conductor_35_49','conductor_50_mas']



atrib_categoricos = ['RANGO_HORARIO', 'DIA_SEMANA', 'TIPO_ACCIDENTE']


target = ['NVICTIM_INTERV']


columnas = atrib_numericos + atrib_categoricos + target

print(columnas)

['MAL_TIEMPO', 'MAL_ESTADO_SUELO', 'num_vehic_implic', 'conductor_menor', 'conductor_18_34', 'conductor_35_49', 'conductor_50_mas', 'RANGO_HORARIO', 'DIA_SEMANA', 'TIPO_ACCIDENTE', 'NVICTIM_INTERV']


### Creamos variables dummies (categoricas a números)

In [9]:
dummies = pd.get_dummies(df_seleccion[atrib_categoricos])

dummies

,RANGO_HORARIO_DE 00:00 A 7:00,RANGO_HORARIO_DE 12:00 A 17:00,RANGO_HORARIO_DE 17:00 A 21:00,RANGO_HORARIO_DE 21:00 A 00:00,RANGO_HORARIO_DE 7:00 A 12:00,DIA_SEMANA_FIN DE SEMANA,DIA_SEMANA_LABORABLE,TIPO_ACCIDENTE_ATROPELLO,TIPO_ACCIDENTE_CAIDA BICICLETA,TIPO_ACCIDENTE_CAIDA MOTOR_2_3_RUEDAS,TIPO_ACCIDENTE_CAIDA VIAJERO BUS,TIPO_ACCIDENTE_CHOQUE CON OBJETO FIJO,TIPO_ACCIDENTE_COLISION DOBLE,TIPO_ACCIDENTE_COLISION MULTIPLE,TIPO_ACCIDENTE_VUELCO_OTROS
0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
86813,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
86814,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
86815,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0


### Creamos el dataset final para la fase de modelado.

In [10]:
df_modelo = pd.concat([df_seleccion[atrib_numericos],  dummies, df_seleccion[target]], axis=1)

print(df_modelo.shape)

df_modelo.sample(10)

(86817, 23)


,MAL_TIEMPO,MAL_ESTADO_SUELO,num_vehic_implic,conductor_menor,conductor_18_34,conductor_35_49,conductor_50_mas,RANGO_HORARIO_DE 00:00 A 7:00,RANGO_HORARIO_DE 12:00 A 17:00,RANGO_HORARIO_DE 17:00 A 21:00,...,DIA_SEMANA_LABORABLE,TIPO_ACCIDENTE_ATROPELLO,TIPO_ACCIDENTE_CAIDA BICICLETA,TIPO_ACCIDENTE_CAIDA MOTOR_2_3_RUEDAS,TIPO_ACCIDENTE_CAIDA VIAJERO BUS,TIPO_ACCIDENTE_CHOQUE CON OBJETO FIJO,TIPO_ACCIDENTE_COLISION DOBLE,TIPO_ACCIDENTE_COLISION MULTIPLE,TIPO_ACCIDENTE_VUELCO_OTROS,NVICTIM_INTERV
17005,1,1,1,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,LEVE
30329,0,0,2,0,0,1,1,0,1,0,...,1,0,0,0,0,0,1,0,0,LEVE
30323,0,0,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,LEVE
44895,1,1,2,0,1,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,LEVE
3578,0,0,1,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,MEDIA/GRAVE
33453,1,1,2,0,1,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,LEVE
24644,1,1,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,LEVE
42539,0,0,2,0,1,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,LEVE
26260,0,0,2,0,0,2,0,0,1,0,...,1,0,0,0,0,0,1,0,0,LEVE
49359,0,0,2,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,LEVE


In [11]:
df_modelo.columns

Index(['MAL_TIEMPO', 'MAL_ESTADO_SUELO', 'num_vehic_implic', 'conductor_menor',
       'conductor_18_34', 'conductor_35_49', 'conductor_50_mas',
       'RANGO_HORARIO_DE 00:00 A 7:00', 'RANGO_HORARIO_DE 12:00 A 17:00',
       'RANGO_HORARIO_DE 17:00 A 21:00', 'RANGO_HORARIO_DE 21:00 A 00:00',
       'RANGO_HORARIO_DE 7:00 A 12:00', 'DIA_SEMANA_FIN DE SEMANA',
       'DIA_SEMANA_LABORABLE', 'TIPO_ACCIDENTE_ATROPELLO',
       'TIPO_ACCIDENTE_CAIDA BICICLETA',
       'TIPO_ACCIDENTE_CAIDA MOTOR_2_3_RUEDAS',
       'TIPO_ACCIDENTE_CAIDA VIAJERO BUS',
       'TIPO_ACCIDENTE_CHOQUE CON OBJETO FIJO',
       'TIPO_ACCIDENTE_COLISION DOBLE', 'TIPO_ACCIDENTE_COLISION MULTIPLE',
       'TIPO_ACCIDENTE_VUELCO_OTROS', 'NVICTIM_INTERV'],
      dtype='object')

### Guardamos finalmente en archivo 'csv'.

In [12]:

df_modelo.to_csv('./madrid_accidentes_modelar_NVICTIM_INTERV_v1_0.csv',sep=',',header=True,index=False,encoding='latin1')